In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pymongo
client = pymongo.MongoClient()
db = client.metrics

In [3]:
import numpy as np
from collections import defaultdict
from joblib import Parallel, delayed
import dill

from Metrics import rpf1
from load_data import load_process_essays
from wordtagginghelper import merge_dictionaries

#from gensim.models import Word2Vec
from window_based_tagger_config import get_config
from IdGenerator import IdGenerator as idGen
from results_procesor import ResultsProcessor, __MICRO_F1__
from Rpfa import micro_rpfa
from collections import defaultdict

import Settings
import logging
import datetime

In [4]:
import pickle
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
training_pickled = settings.data_directory + "CoralBleaching/Thesis_Dataset/training.pl"
models_folder = root_folder + "Models/Bi-LSTM/"

config = get_config(training_folder)
processor = ResultsProcessor()

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [5]:
with open(training_pickled, "rb+") as f:
    tagged_essays = pickle.load(f)
len(tagged_essays)

902

In [6]:
import datetime, logging
print("Started at: " + str(datetime.datetime.now()))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()

Started at: 2017-07-08 23:51:14.627096


In [7]:
CAUSER = "Causer"
RESULT = "Result"
EXPLICIT = "explicit"
CAUSER_EXPLICIT = "Causer_Explicit"
EXPLICIT_RESULT = "Explicit_Result"
CAUSER_EXPLICIT_RESULT = "Causer_Explicit_Result"
CAUSER_RESULT = "Causer_Result"

In [8]:
tag_freq = defaultdict(int)
unique_words = set()
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            unique_words.add(word)
            for tag in tags:
                tag_freq[tag] += 1

EMPTY_TAG = "Empty"
#TODO - don't ignore Anaphor, other and rhetoricals here
cr_tags = list((t for t in tag_freq.keys() if ( "->" in t) and not "Anaphor" in t and not "other" in t and not "rhetorical" in t))
regular_tags = list((t for t in tag_freq.keys() if t[0].isdigit()))

vtags = set(regular_tags)
#vtags.add(EMPTY_TAG)
#cr_tags = vtags

len(unique_words)

1641

In [9]:
regular_tags

['50', '5', '4', '11', '13', '3', '7', '1', '6', '14', '2', '12', '5b']

In [10]:
cr_tags

['Causer:5->Result:50',
 'Causer:7->Result:50',
 'Causer:3->Result:4',
 'Causer:1->Result:50',
 'Causer:11->Result:50',
 'Causer:13->Result:50',
 'Causer:6->Result:50',
 'Causer:3->Result:5',
 'Causer:4->Result:14',
 'Causer:3->Result:1',
 'Causer:1->Result:3',
 'Causer:3->Result:50',
 'Causer:4->Result:5',
 'Causer:3->Result:7',
 'Causer:6->Result:7',
 'Causer:11->Result:12',
 'Causer:12->Result:13',
 'Causer:7->Result:5b',
 'Causer:5b->Result:50',
 'Causer:50->Result:50',
 'Causer:1->Result:2',
 'Causer:2->Result:3',
 'Causer:11->Result:3',
 'Causer:11->Result:13',
 'Causer:11->Result:14',
 'Causer:5->Result:5b',
 'Causer:1->Result:7',
 'Causer:7->Result:14',
 'Causer:5->Result:7',
 'Causer:11->Result:6',
 'Causer:13->Result:14',
 'Causer:3->Result:13',
 'Causer:3->Result:14',
 'Causer:4->Result:50',
 'Causer:14->Result:50',
 'Causer:1->Result:14',
 'Causer:4->Result:5b',
 'Causer:1->Result:4',
 'Causer:6->Result:14',
 'Causer:5b->Result:7',
 'Causer:3->Result:5b',
 'Causer:13->Resul

In [11]:
[k for k in sorted(tag_freq.keys()) if "->" not in k]

['1',
 '11',
 '12',
 '13',
 '14',
 '2',
 '3',
 '4',
 '5',
 '50',
 '5b',
 '6',
 '7',
 'Anaphor',
 'COMPILED',
 'Causer',
 'Causer:1',
 'Causer:11',
 'Causer:12',
 'Causer:13',
 'Causer:14',
 'Causer:2',
 'Causer:3',
 'Causer:4',
 'Causer:5',
 'Causer:50',
 'Causer:5b',
 'Causer:6',
 'Causer:7',
 'Causer:Anaphor',
 'Causer:other',
 'Causer:rhetorical',
 'Result',
 'Result:1',
 'Result:11',
 'Result:12',
 'Result:13',
 'Result:14',
 'Result:2',
 'Result:3',
 'Result:4',
 'Result:5',
 'Result:50',
 'Result:5b',
 'Result:6',
 'Result:7',
 'Result:Anaphor',
 'Result:other',
 'Result:rhetorical',
 'explicit',
 'other',
 'rhetorical']

In [12]:
START = "<start>"
END   = "<end>"

def get_training_data(tessays):
    # outputs
    xs = []
    ys = []
    ys_bytag = defaultdict(list)
    ys_bytag_sent = defaultdict(list)
    seq_lens = []

    # cut texts after this number of words (among top max_features most common words)
    for essay in tessays:
        for sentence in essay.sentences:
            row = []
            y_found = False
            y_seq = []
            unique_tags = set()
            for word, tags in [(START, set())] + sentence + [(END, set())]:
                id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
                row.append(id)                
                # remove unwanted tags
                tags = vtags.intersection(tags)
                unique_tags.update(tags)
                # retain all tags for evaluation (not just most common)
                # SKIP the START and END tags
                if word != START and word != END:
                    for t in (vtags - set([EMPTY_TAG])):
                        if t in tags:
                            ys_bytag[t].append(1)
                        else:
                            ys_bytag[t].append(0)

                # encode ys with most common tag only
                if len(tags) > 1:
                    most_common = max(tags, key=lambda t: tag_freq[t])
                    tags = set([most_common])
                if len(tags) == 0:
                    tags.add(EMPTY_TAG)

                one_hot = []
                for t in vtags:
                    if t in tags:
                        one_hot.append(1)
                    else:
                        one_hot.append(0)
                y_seq.append(one_hot)
            
            for tag in vtags:
                if tag in unique_tags:
                    ys_bytag_sent[tag].append(1)
                else:
                    ys_bytag_sent[tag].append(0)
                
            seq_lens.append(len(row)-2)
            ys.append(y_seq)
            xs.append(row)
    
    xs = np.asarray(xs)
    ys = np.asarray(ys)
    assert xs.shape[0] == ys.shape[0], "Sequences should have the same number of rows"
    assert xs.shape[1] == ys.shape[1] == maxlen, "Sequences should have the same lengths"
    return xs, ys, ys_bytag, ys_bytag_sent, seq_lens

## <span style="color:red">How to Solve for Sentences with Same Cause, Multiple Effects?</span>
* Dependency tree requires parent child relationship
* One cause, multiple effects means 1 child with multiple parents
* **Solution** - maybe link effects to effects in this scenario?

## Feature Extraction

In [13]:
WINDOW_SIZE = 7

config["window_size"] = WINDOW_SIZE
offset = int((config["window_size"] - 1) / 2)
offset

3

In [14]:
def attach_function_identifier(fn, d):
    try:
        s = fn.func_name + "["
    except:
        s = fn.__name__ + "["
        fn.func_name = fn.__name__

    for k, v in sorted(d.items(), key = lambda tpl: tpl[0]):
        if k == fn.func_name:
            continue
        if type(v) == dict:
            continue
        s +=   "%s:%s " % (str(k), str(v))
    fn.func_name = s.strip() + "]"
    return fn

def fact_extract_positional_word_features_stemmed(offset):
    """ offset      :   int
                            the number of words either side of the input to extract features from
        returns     :   fn
                            feature extractor function: FeatureExtactorInput -> dict
    """
    lcls = locals()
    # curry offset
    def fn_pos_wd_feats_stemmed(input, val=1):
        return extract_positional_word_features_stemmed(offset, input, val)
    return attach_function_identifier(fn_pos_wd_feats_stemmed, lcls) # recently renamed for mongodob logging

def extract_positional_word_features_stemmed(offset, input, val = 1):
    """ offset      :   int
                           the number of words either side of the input to extract features from
        input      :    FeatureExtactorInput
                            input to feature extractor
        returns     :   dict
                            dictionary of features
    """

    feats = {}
    start = input.wordix - offset
    stop  = input.wordix + offset

    end = len(input.sentence) - 1
    for i in range(start, stop+1):
        relative_offset = str(i - input.wordix)
        if i < 0:
            feats["WD" +__START__ + ":" + relative_offset] = val
        elif i > end:
            feats["WD" +__END__ + ":" + relative_offset] = val
        else:
            offset_word = stem(input.sentence[i])
            feats["WD:" + relative_offset + "->" + offset_word] = val
    return feats

def fact_extract_ngram_features_stemmed(offset, ngram_size):
    """ offset      :   int
                            the number of words either side of the input to extract features from
        ngram_size  :   int
                            the size of the ngrams
        returns     :   fn
                            feature extractor function: FeatureExtactorInput -> dict
    """
    lcls = locals()
    # curry offset and ngram size
    def fn_pos_ngram_feat_stemmed(input, val=1):
        return extract_ngram_features_stemmed(offset, ngram_size, input, val)
    return attach_function_identifier(fn_pos_ngram_feat_stemmed, lcls)

def extract_ngram_features_stemmed(offset, ngram_size, input, val = 1):
    """ offset      :   int
                           the number of words either side of the input to extract features from
        ngram_size  :   int
                            the size of the ngrams
        input      :    FeatureExtactorInput
                            input to feature extractor
        returns     :   dict
                            dictionary of features
    """

    feats = {}
    end = len(input.sentence) - 1

    # fix to within bounds only
    start = max(0, input.wordix - offset)
    stop  = min(end, input.wordix + offset)

    window = list(input.sentence[start:stop+1])
    window = list(map(stem, window))
    if input.wordix < offset:
        diff = offset - input.wordix
        for i in range(diff):
            window.insert(0,__START__)
    if input.wordix + offset > end:
        diff = input.wordix + offset - end
        for i in range(diff):
            window.append(__END__)

    ngrams = compute_ngrams(window, ngram_size, ngram_size)
    str_num_ngrams = str(ngram_size)

    for i, offset_ngram in enumerate(ngrams):
        relative_offset = str(i - offset)
        str_ngram = ",".join(offset_ngram)
        feats["POS_" + str_num_ngrams + "GRAMS:" + relative_offset + "->" + str_ngram] = val

    return feats

def fact_extract_bow_ngram_features(offset, ngram_size):
    """ offset      :   int
                            the number of words either side of the input to extract features from
        ngram_size  :   int
                            the size of the ngrams
        returns     :   fn
                            feature extractor function: FeatureExtactorInput -> dict
    """
    # curry offset and ngram size
    lcls = locals()
    def fn_bow_ngram_feat(input, val=1):
        return extract_bow_ngram_features(offset, ngram_size, input, val)
    return attach_function_identifier(fn_bow_ngram_feat, lcls)

def extract_bow_ngram_features(offset, ngram_size, input, val = 1):
    """ offset      :   int
                           the number of words either side of the input to extract features from
        ngram_size  :   int
                            the size of the ngrams
        input      :    FeatureExtactorInput
                            input to feature extractor
        returns     :   dict
                            dictionary of features
    """

    feats = {}
    end = len(input.sentence) - 1

    # fix to within bounds only
    start = max(0, input.wordix - offset)
    stop  = min(end, input.wordix + offset)

    window = list(input.sentence[start:stop+1])
    if input.wordix < offset:
        diff = offset - input.wordix
        for i in range(diff):
            window.insert(0,__START__)
    if input.wordix + offset > end:
        diff = input.wordix + offset - end
        for i in range(diff):
            window.append(__END__)

    ngrams = compute_ngrams(window, ngram_size, ngram_size)
    str_num_ngrams = str(ngram_size)

    for i, offset_ngram in enumerate(ngrams):
        str_ngram = ",".join(offset_ngram)
        feats["POS_" + str_num_ngrams + "GRAMS:BOW" + "->" + str_ngram] = val

    return feats

In [15]:
__START__ = "<START>"
__END__   = "<END>"

#from featureextractionfunctions \
    #import fact_extract_positional_word_features_stemmed, fact_extract_ngram_features_stemmed,\
    #fact_extract_bow_ngram_features, extract_brown_cluster    
    #extract_dependency_relation

unigram_window_stemmed  = fact_extract_positional_word_features_stemmed(offset)
biigram_window_stemmed  = fact_extract_ngram_features_stemmed(offset, 2)
triigram_window_stemmed = fact_extract_ngram_features_stemmed(offset, 3)
unigram_bow_window      = fact_extract_bow_ngram_features(offset, 1)

#TODO - add in full feature set, but keep simple for now
#optimal CB feature set
extractors = [
    unigram_window_stemmed,
    biigram_window_stemmed,
    #triigram_window_stemmed,
    #unigram_bow_window,
    #extract_dependency_relation,
    #extract_brown_cluster
]

In [16]:
from Decorators import memoize
from nltk import PorterStemmer
from NgramGenerator import compute_ngrams
from featureextractortransformer import FeatureExtractorTransformer

stemmer = PorterStemmer()

@memoize
def stem(word):
    return stemmer.stem(word)

feature_extractor = FeatureExtractorTransformer(extractors)
essay_feats = feature_extractor.transform(tagged_essays)
len(tagged_essays), len(essay_feats)

(902, 902)

In [17]:
set(range(10)) - set(range(5))

{5, 6, 7, 8, 9}

In [18]:
class SearnModel(object):
    Shift = "Shift"
    Reduce = "Reduce"
    Left_ARC = "LeftArc"
    Right_ARC = "RightArc"
    Root = "Root"

    CAUSAL = set([CAUSER, EXPLICIT, RESULT])
    EMPTY_TAG = "Empty"
    
    def __init__(self, feat_extractor, tags, tag_freq, cr_tags, base_learner_fact, beta):
        # init checks
        #assert CAUSER in tags, "%s must be in tags" % CAUSER
        #assert RESULT in tags, "%s must be in tags" % RESULT
        #assert EXPLICIT in tags, "%s must be in tags" % EXPLICIT

        self.feat_extractor = feature_extractor    # feature extractor (for use later)
        self.base_learner_fact = base_learner_fact # Sklearn classifier
        self.tags = set(tags)                      # tags for basic tagging
        self.tag_freq = tag_freq                   # frequency of word tags
        
        self.cr_tags = set(cr_tags)                # causal relation tags
            
        self.actions = set([SearnModel.Shift, SearnModel.Reduce, SearnModel.Skip])
        self.epoch = -1
        self.beta = beta
        self.stack = []
        self.tagging_models = {}
        self.parser_model = None
        
    def train(self, essay_feats, epochs):
        #essay_feats = self.feat_extractor.transform(tagged_essays)        
        for i in range(0, epochs):
            self.epoch +=1
            tagging_models = {}
            examples_with_loss = dict() # dict of tag\decision to examples (with labels)
            
            for essay_ix, essay in enumerate(essay_feats):
                for sent_ix, taggged_sentence in enumerate(essay.sentences):
                    exs = self.taggged_sentence(taggged_sentence)                    
    
    def tag_sentence(self, taggged_sentence):
        prediction_history = []
        ground_truth_history = []
        previous_words = []
        self.stack = [Root]
        
        tagging_examples = []
        parsing_examples = []
        arcs = set()

        gt_cr_tags = set()
        for i, (wd) in enumerate(taggged_sentence):
            gt_cr_tags.update(wd.tags)
        gt_cr_tags = gt_cr_tags.intersection(self.cr_tags)
        
        for i, (wd) in enumerate(taggged_sentence):
            #TODO - decay beta to determine when to return GT and when not
            return_ground_truth = True
            
            gt_tags = wd.tags.intersection(self.tags)
            gt_cr_tags = wd.tags.intersection(self.cr_tags)
            
            current_word = wd.word
            # Get tagging features            
            window_feats = dict(wd.features.items()) # - Don't mutate the feat dictionary
            tag_hist_feats = self.get_conditional_feats(current_word, previous_words, ground_truth_history)
            tag_feats = tag_hist_feats.update(window_feats)
            # Tag Word
            predicted_tag = self.tag_word(self, gt_tags, tag_feats, return_ground_truth)
            for gt_tag in gt_tags:
                #TODO - fill in the loss here
                tagging_examples.append((tag_feats, gt_tag, 1.0)) #xs, y, LOSS
                
            if self.__is_new_tag__(predicted_tag):
                parse_feats = dict(tag_feats.items())
                parse_feats["WORD_CONCEPT_TAG:" + str(predicted_tag)] = 1
                
                # has to come before parse_word, as that modifies the stack
                oracle_decision = self.parse_word_oracle(predicted_tag, gt_cr_tags)
                #TODO - fill in the loss here
                parsing_examples.append((parse_feats, oracle_decision, 1.0))
                #Modifies the stack
                parse_decision = self.parse_word(predicted_tag, parse_feats, oracle_decision, return_ground_truth, arcs)
                parse_decisions = []
                # Left_Arc and Reduce don't advance the buffer
                while parse_decision in [Left_ARC, Reduce]:
                    #TODO  What ground truth history do we add here?
                    parse_feats = self.get_conditional_feats(current_word, previous_words, ground_truth_history)
                    parse_feats.update(window_feats)
                    parse_feats["WORD_CONCEPT_TAG:" + str(predicted_tag)] = 1
                    
                    pass
                pass
                
            previous_words.append(wd.word)
            ground_truth_history.append(gt_tags.union(gt_cr_tags))            
            
            ## parse decisions
            new_tags = set([predicted_tag])
            #TODO - get parser decisions
            parser_tags = set()
            new_tags.update(parser_tags)
            prediction_history.append(new_tags)
    
    def get_conditional_feats(self, current_word, prev_words, history):
        feats = {}
        if len(history) > 0:
            #TODO add feats for tags not predicted and actions not taken
            prev_tags = history[-1]
            for t in prev_tags:
                feats["tag-1:" + t] = 1
                feats["tag-1:" + t + " wd:"  + current_word] = 1
                feats["tag-1:" + t + " wd-1:"+ prev_words[-1]] = 1
                
            if len(history) > 1:
                prev_prev_tags = history[-2]
                for t in prev_prev_tags:
                    # tag bigrams
                    for prev_tag in prev_tags:
                        feats["tag-1:" + prev_tag + " tag-2:" + t] = 1
                    feats["tag-2:" + t] = 1
                    feats["tag-2:" + t + " wd:"   + current_word] = 1
                    feats["tag-2:" + t + " wd-2:" + prev_words[-2]] = 1
        
        for i in range(4):
            offset = -(i+1)
            if len(self.stack) < abs(offset):
                break
            s1_key = "s" + str(offset) + ":" + self.stack[offset]
            feats[s1_key] = 1
            # Stack Bigram feats
            if len(self.stack) >= abs(offset) + 1:
                next_offset = offset-1
                s2_key = "s" + str(next_offset) + ":" + self.stack[next_offset]
                feats[s1_key + " " + s2_key] = 1
        return feats
    
    def tag_word(self, ground_truth_tags, tag_feats, return_ground_truth):
        tags = None
        if return_ground_truth:
            tags = self.tags.intersection(ground_truth_tags)            
        else:
            #TODO - implement word tagging model
            raise Exception("#TODO - implement word tagger")
        if not tags:
            return EMPTY_TAG
        most_freq = max(tags, key = lambda tag: self.tag_freq[tag])
        return most_freq
    
    def __is_new_tag__(self, predicted_tag, history):
        if predicted_tag == EMPTY_TAG:
            return False
        if len(history) == 0 or predicted_tag not in history[-1]:
            return True
        return False
    
    def parse_word(self, predicted_tag, parse_feats, oracle_decision, return_ground_truth, arcs):
        parse_decision = None
        if return_ground_truth:
            parse_decision = oracle_decision
        else:
            #TODO - implement ML parser
            raise Exception("#TODO - implement parser")
        if parse_decision == Shift:
            self.stack.append(predicted_tag)
        elif parse_decision == Reduce:
            self.stack.pop()
        elif parse_decision == Left_ARC:
            tos = self.stack.pop()
            arc = [tos,predicted_tag] # Cause -> Effect  
            arcs.add(arc)
        elif parse_decision == Right_ARC:
            arc = [predicted_tag, self.stack[-1]] # Effect <- Cause so ["Cause Code","Effect Code"]
            arcs.add(arc)
            self.stack.append(predicted_tag)
        return parse_decision
        
    def parse_word_oracle(self, predicted_tag, golden_tags, prediction_history):
        csl_tags = self.cr_tags.intersection(golden_tags)
        tos = self.stack[-1]
        #TODO - about 5% of causal relations lack a constituent cause, effect or both - remove these from training?
        #TODO - if no causal in the sentence, shift, reduce, shift, reduce, etc
        #TODO - if causal in sentence, but this code not in causal, shift, reduce
        #TODO - or do we implement a second classifier - Causal, explicit, Result
        if len(prediction_history) == 0 or len(csl_tags) == 0:
            return Shift
        pairs = []
        for crel in csl_tags:
            left, right = crel.replace("Causer:","").replace("Result:","").split("->")
            pairs.append((left,right))
        
        if len(pairs) == 1:
            left, right = pairs[0]
            if tos == left and predicted_tag == right:
                return Left_ARC
            elif tos == right and predicted_tag == left:
                return Right_ARC
        
        #TODO - get optimal parse decision given current state
        #TODO - enforce valid decisions
        return None

In [19]:
def normalize(code):
    if not code.endswith("b"):
        code += " "
    return code.rjust(3, "0")

def extract_lr(cr):
    return cr.replace("Causer:","").replace("Result:","").split("->")

def normalize_cr(cr):
    pair = extract_lr(cr)
    return (normalize(pair[0]),normalize(pair[1]))

normalize_cr("Causer:5->Result:50")

('05 ', '50 ')

In [20]:
normPair2 = {}
for crel in cr_tags:
    pair = normalize_cr(crel)
    normPair2[pair] = crel

In [21]:
pairs = list(map(normalize_cr, cr_tags))
len([p for p in pairs if p[0] <  p[1]]),len([p for p in pairs if p[0] >  p[1]]),

(52, 28)

In [22]:
total_lt = 0
total_gt = 0
for p in pairs:
    freq = tag_freq[normPair2[p]]
    if p[0] >  p[1]:        
        total_gt += freq
    else:
        total_lt += freq
print(total_gt, total_lt)

759 33148


In [23]:
total_gt/(total_gt + total_lt)

0.022384758309493616

In [24]:
def normalize_cr(cr):
    return cr.replace("Causer:","").replace("Result:","")

lr_tags = set()
for crel in cr_tags:
    l,r = crel.split("->")
    lr_tags.add(l)
    lr_tags.add(r)
    l,r = normalize_cr(crel).split("->")
    lr_tags.add(l)
    lr_tags.add(r)
sorted(lr_tags)

['1',
 '11',
 '12',
 '13',
 '14',
 '2',
 '3',
 '4',
 '5',
 '50',
 '5b',
 '6',
 '7',
 'Causer:1',
 'Causer:11',
 'Causer:12',
 'Causer:13',
 'Causer:14',
 'Causer:2',
 'Causer:3',
 'Causer:4',
 'Causer:5',
 'Causer:50',
 'Causer:5b',
 'Causer:6',
 'Causer:7',
 'Result:1',
 'Result:11',
 'Result:12',
 'Result:13',
 'Result:14',
 'Result:2',
 'Result:3',
 'Result:4',
 'Result:5',
 'Result:50',
 'Result:5b',
 'Result:6',
 'Result:7']

In [25]:
len(lr_tags), len(cr_tags)

(39, 82)

# Get CRels and Theirs Tags for Testing Oracle

* Here we have the concept of 'Supported Relations'
* A number of the causal relations do not have both components of the causal relation tagged in the sentence
* For training the parser, we need to only train it on these sanitized relations, and where the causer or result tag
 is not prefixed with the "Causer:..." or "Result:..." then we need to ammend this for training purposes
* Note that when we evaluate the parser, we use the original dataset, thus this parser will be unable to learn these removed relations (but RQ 3 and 4 should address that)

### Gather Causal Relations and Supporting Causer \ Result Codes

In [26]:
from pprint import pprint

relations = []
unq_cr_tags = set(cr_tags)
skipped_sent = 0
skipped_crels = 0
num_sents = 0
for essay_ix, essay in enumerate(tagged_essays):
    for sent_ix, taggged_sentence in enumerate(essay.sentences):
        num_sents += 1
        tag_seq = []
        un_tags = set()
        crel2tags = defaultdict(set)
        def add_tag(tag, crel):
            if tag not in un_tags:
                tag_seq.append(tag)
                un_tags.add(tag)
            crel2tags[crel].add(tag)
        
        has_causal = False
        for i, (wd,tags) in enumerate(taggged_sentence):
            csl = unq_cr_tags.intersection(tags)
            if not csl:
                continue
            has_causal = True
            for crel in csl:
                l_causer, r_effect = crel.split("->")
                l,r = normalize_cr(crel).split("->")
                if l_causer in tags:
                    add_tag(l, crel)
                if r_effect in tags:
                    add_tag(r, crel)                
                if l in tags:
                    add_tag(l, crel)
                if r in tags:
                    add_tag(r, crel)
        
        # Don't count sentences without any relations as skipped
        if not has_causal:
            continue
        
        supported_causal = set()
        supported_codes = set()
        for crel, tags in crel2tags.items():
            if len(tags) < 2:
                skipped_crels += 1
                continue
            supported_causal.add(crel)
            supported_codes.update(tags)
        
        if not supported_causal:
            skipped_sent += 1
            continue
        # filter out any tags that were only part of unsupported causal relations
        tag_seq = [tag for tag in tag_seq if tag in supported_codes]
        relations.append((supported_causal,tag_seq))
        
num_sents, len(relations), skipped_sent, skipped_crels

(8292, 2217, 86, 141)

In [27]:
print("[")
for supported_causal, tag_seq in relations:
    if len(supported_causal) > 1:
        print("(" +str(sorted(supported_causal)) + "," + str(tag_seq) + "),")
print("]")

[
(['Causer:1->Result:50', 'Causer:11->Result:50', 'Causer:13->Result:50'],['50', '13', '1', '11']),
(['Causer:1->Result:50', 'Causer:3->Result:1'],['50', '1', '3']),
(['Causer:13->Result:50', 'Causer:3->Result:50'],['3', '50', '13']),
(['Causer:3->Result:7', 'Causer:7->Result:50'],['3', '7', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13', 'Causer:13->Result:50'],['11', '12', '13', '50']),
(['Causer:5b->Result:50', 'Causer:7->Result:5b'],['7', '5b', '50']),
(['Causer:3->Result:7', 'Causer:7->Result:50'],['3', '7', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:11->Result:13', 'Causer:11->Result:3', 'Causer:11->Result:50'],['11', '3', '13', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:1->Result:50', 'Causer:1->Result:7', 'Causer:3->Result:50', 'Causer:3->Result:7'],['1', '3', '50', '7']),
(['Causer:7->Result:14', 'Causer:7->Result:50'],

In [28]:
#e = tagged_essays[538]
#e.sentences[9]

## Note You Can Set Per Observation Weights in XGBoost
* http://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training
  * see weight parameter in dtrain

## TODO
* Need to make sure the tagger tags EXCPLICT tags. These can then be skipped by the parser, but will be included in the features used to train the parser and taggger
